Notebook che definisce un autoencoder standard per ridurre le feature di MNIST. Per semplicità, ho caricato sulla repository l'encoder già addestrato per ridurre la dimensionalità a 64. È quindi sufficiente eseguire le celle che caricano "autoencoder64ReLU.pth" e che salvano i nuovi dataset, così da poterli utilizzare sull'altro notebook.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
# Trasformazioni: converto in tensori e normalizzo
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.reshape(-1))])

# Scarico i dataset
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

encoding_dim = 64

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, encod_dim):
        super().__init__()
        # Encoder: da 784 → encoding_dim
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, encod_dim),  # Dimensione compressa
            nn.ReLU()
        )
        
        # Decoder: da encod_dim → 784
        self.decoder = nn.Sequential(
            nn.Linear(encod_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()  # Output tra 0 e 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def extra_repr(self) -> str:
        return f"in_features={784}, out_features={encoding_dim}"

In [ ]:
autoencoder = Autoencoder(encoding_dim).to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    size = len(train_dataloader.dataset)
    autoencoder.train()
    for batch, (images, _) in enumerate(train_dataloader):
        images = images.to(device)
        
        # Forward
        outputs = autoencoder(images)
        loss = criterion(outputs, images) 
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(images)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


In [ ]:
fig , ax = plt.subplots(2, 6, figsize = (16,6))

autoencoder.eval()
with torch.no_grad():
    for i, (img, _)  in enumerate(train_dataloader):
        x = img[0].to(device)
        pred = autoencoder(x)
        ax[0,i].imshow(x.cpu().reshape(28,28)) 
        ax[1,i].imshow(pred.cpu().reshape(28,28))
        if i==5:
            break
    plt.show()

In [ ]:
torch.save(autoencoder.state_dict(), "autoencoder64ReLU.pth")
print("Saved PyTorch Model State to autoencoder64ReLU.pth")

In [ ]:
autoencoder = Autoencoder(encoding_dim).to(device)
autoencoder.load_state_dict(torch.load("autoencoder64ReLU.pth", weights_only=True))
print(autoencoder)

In [ ]:
def encode_images(data_loader, model, padding = 0, dtype = torch.float64, device = 'cpu'):
    model.eval()
    compressed_data = []
    labels = []
    
    with torch.no_grad():
        for images, y in data_loader:
            images = images.to(device)
            model = model.to(device)
            compressed = model.encoder(images)
            if padding:
                compressed = torch.cat(
                    [compressed.type(dtype),
                    torch.zeros(compressed.shape[0], padding, device = device, dtype = dtype)
                    ],
                    dim = 1)   
            labels.append(y.to(device))        
            compressed_data.append(compressed)

    return torch.cat(compressed_data, dim=0), torch.cat(labels, dim=0)

In [ ]:
torch.save(encode_images(train_dataloader, autoencoder, padding = 10, dtype = torch.complex64), "qsw_train_data_encoded64.pt")
torch.save(encode_images(test_dataloader, autoencoder, padding = 10, dtype = torch.complex64), "qsw_test_data_encoded64.pt")